In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import openpyxl
import re
import os
from variableUtils import *
import variableUtils
from Utils import *
from ClassUtils import *
from pprint import pprint
import json
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from reportlab.lib.pagesizes import letter, landscape, A4, A3
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, PageBreak, Paragraph, Spacer, Image
from reportlab.lib import colors
from matplotlib.backends.backend_pdf import PdfPages
from reportlab.platypus import Paragraph, Spacer, KeepTogether, KeepInFrame
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch

from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.formatting.rule import FormulaRule


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sklearn.cluster import KMeans
from dotenv import load_dotenv
load_dotenv()
from openai import OpenAI
from openpyxl import Workbook
client = OpenAI()
from sentence_transformers import SentenceTransformer, util
import numpy as np
from textblob import TextBlob
vectorizer = TfidfVectorizer()


warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print(sns.__version__)


In [ ]:
folder = 'Extra\MCQ Creation'
files = os.listdir(folder)
dfDict = {}
colTextDict = {}
fullDf = pd.DataFrame(columns=['Question', 'Total Score', 'Section', 'Year', 'Question Code'])
for file in files[:3]:
    print(file)
    if not file.endswith('.csv'):
        continue
    year = file.split(' ')[-1].split('.')[0]
    questionDf = pd.DataFrame(columns = ['Question', 'Total Score', 'Section', 'Year', 'Question Code'])
    df = pd.read_csv(f'{folder}/{file}')
    # for i, col in enumerate(df.columns):
    #     print(i, col)
    # display(df.head(2))
    columns = df.columns.to_list()
    sectionACols = columns[10:90:2]
    sectionBCols = columns[92:171:2]
    print(f'# Section A columns: {len(sectionACols)}, {sectionACols}')
    print(f'# Section B columns: {len(sectionBCols)}, {sectionBCols}')
    # questionCols = [col for col in df.columns if re.match(r'\d+:', col)]
    # questionCols =questionCols[1:]
    questionCols = sectionACols + sectionBCols
    print(f'# questions: {len(questionCols)} || {questionCols}')
    # markCols = df.columns.to_list()[11::2]
    sectionAMarks = columns[11:91:2]
    sectionBMarks = columns[93:172:2]
    markCols = sectionAMarks + sectionBMarks
    df[markCols] = df[markCols].apply(pd.to_numeric, errors='coerce')
    print(f'# marks: {len(markCols)} || {markCols}')
    totalSeries = df[markCols].sum(axis = 0, numeric_only=True)
    df.loc['Column Total'] = totalSeries
    # display(df.tail(3))
    for i, col in enumerate(questionCols):
        colText = col.split(':')[1]
        colCode = f'Q{i+1}'
        colTextDict[(colCode, year)] = colText
        questionDf.loc[i] = [colText, totalSeries[i]/len(df), None, year, colCode]
        if col in sectionACols:
            questionDf.loc[i, 'Section'] = 'A'
        elif col in sectionBCols:
            questionDf.loc[i, 'Section'] = 'B'
    print(len(colTextDict))
    # for i, col in enumerate(sectionACols):
    #     colText = col.split(':')[1]
    #     questionDf.loc[i] = [colText, totalSeries[i], 'A']
    
    # for i, col in enumerate(sectionBCols):
    #     colText = col.split(':')[1]
    #     questionDf.loc[i] = [colText, totalSeries[i], 'B']
    
    questionDf['Year'] = year
    # display(questionDf)
    fullDf = pd.concat([fullDf, questionDf], axis = 0)

    # break
display(fullDf)

In [ ]:


# Initialize the model
model = SentenceTransformer('all-MiniLM-L6-v2')  # Smaller model, effective for sentence similarity

def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding


def getSimilarityMatrix(embeddings):
    # Calculate similarity matrix
    similarity_matrix = util.pytorch_cos_sim(embeddings, embeddings).numpy()
    return similarity_matrix


fullDf['Embedding'] = fullDf['Question'].apply(get_embedding)

In [ ]:
fullDf.sort_values(by = ['Section','Total Score'], ascending = [True, False], inplace = True)
fullDf.reset_index(drop = True, inplace = True)
display(fullDf)
fullDf.to_csv(f'{folder}/Full MCQ.csv', index = False)

In [ ]:
# import ast
# df = pd.read_csv(f'{folder}/Full MCQ.csv')
# # parse string to array
# df['Embedding'] = df['Embedding'].apply(lambda x: ast.literal_eval(x))
# # convert to numpy array
# df['Embedding'] = df['Embedding'].apply(lambda x: np.array(x))
# print(df['Embedding'][0].dtype)

In [ ]:

# fullDf['Embedding'] = fullDf['Embedding'].apply(lambda x: np.array(x))
# fullDf = df.copy()
# print(fullDf['Embedding'].dtype)
# print(fullDf['Embedding'][0].dtype)
def createHist(data, ax, title):
    sns.histplot(data = data, x = 'Total Score', kde = True, ax = ax, bins = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1])
    mean = data['Total Score'].mean()
    median = data['Total Score'].median()
    plt.axvline(mean, color='r', linestyle='--')
    plt.axvline(median, color='g', linestyle='-')
    ax.text(0.5, 0.5, f'Mean: {mean:.2f}', rotation=0, transform=ax.transAxes)
    ax.text(0.5, 0.55, f'Median: {median:.2f}', rotation=0, transform=ax.transAxes)
    plt.title(title)
    plt.show()

fig, ax = plt.subplots(figsize=(10, 6))
createHist(fullDf, ax, 'Overall')


# do for each year
years = fullDf['Year'].unique()
for year in years:
    yearDf = fullDf[fullDf['Year'] == year]
    fig, ax = plt.subplots(figsize=(10, 6))
    createHist(yearDf, ax, year)
    # break

In [ ]:
def selectMostDissimilar(similarityMatrix, numQuestions=4):
    n = similarityMatrix.shape[0]
    # Ensure the diagonal of similarityMatrix is 0 to avoid self-similarity
    # np.fill_diagonal(similarityMatrix, 0)
    
    # Initialize selected questions with the most dissimilar pair
    first, second = np.unravel_index(np.argmin(similarityMatrix), similarityMatrix.shape)
    selected = {first, second}
    
    while len(selected) < numQuestions:
        maxDissimilarity = -np.inf
        nextQuestion = None
        for i in range(n):
            if i not in selected:
                # Calculate dissimilarity as the minimum similarity with the selected set
                dissimilarity = sum(similarityMatrix[i][j] for j in selected)
                if dissimilarity > maxDissimilarity:
                    maxDissimilarity = dissimilarity
                    nextQuestion = i
        selected.add(nextQuestion)
    
    return list(selected)

selectedDf = pd.DataFrame()
for section in fullDf['Section'].unique():
    sectionDf = fullDf[fullDf['Section'] == section]
    for i in range(0, 10):
        data = sectionDf.iloc[i*12:(i+1)*12]
        print(len(data))
        display(data)
        # select four questions with most dissimilarities in embedding
        similarity_matrix = getSimilarityMatrix(data['Embedding'].to_list())
        df = pd.DataFrame(similarity_matrix, columns=[f"Q{i+1}" for i in range(12)], index=[f"Q{i+1}" for i in range(12)])
        display(df)
        selected = selectMostDissimilar(similarity_matrix, 4)
        print(selected)
        selectedData = data.iloc[selected]
        display(selectedData)
        selectedDf = pd.concat([selectedDf, selectedData], axis = 0)
        #



In [ ]:
selectedDf.reset_index(drop = True, inplace = True)

fig, ax = plt.subplots(figsize=(10, 6))
createHist(selectedDf, ax, 'Selected Questions')
fig.savefig(f'{folder}/Selected Questions.png')
selectedDf.drop(columns = ['Embedding'], inplace = True, errors='ignore')
selectedDf.sort_values(by = ['Section','Year', 'Question Code'], ascending = [True, True, True], inplace = True)
display(selectedDf)
# selectedDf.to_csv(f'{folder}/Selected MCQ.csv', index = False, encoding = 'utf-8-sig')

### Language Analysis

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
# Convert dictionary to a list of comments and a matrix of embeddings
def getCluster(df, colEmbedding):
    embeddings = np.array(list(df[colEmbedding].values))
    # Choose the number of clusters
    linked = linkage(embeddings, 'ward')
    # get the appropriate number of clusters from the dendrogram
    # dendrogram(linked, orientation='top', distance_sort='descending', show_leaf_counts=True)
    dendrogram(linked, truncate_mode='lastp', p=10)
    plt.show()
    num_clusters = int(input('Enter the number of clusters: '))
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans.fit(embeddings)
    clusters = kmeans.labels_
    df['Cluster'] = clusters

    wcss = []
    ran = range(1, 21)
    for i in ran:
        kmeans = KMeans(n_clusters=i, random_state=42)
        kmeans.fit(embeddings)
        wcss.append(kmeans.inertia_)

    # Plot the Elbow Method
    plt.figure(figsize=(8, 6))
    plt.plot(ran, wcss, marker='o')
    plt.title('Elbow Method')
    plt.xlabel('Number of Clusters')
    plt.ylabel('WCSS')
    plt.show()

    return df
